In [1]:
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import copy
import multiprocessing as mp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from scipy import sparse, spatial
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

/root/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
tic = time.time()
DATA_PATH = '../pkl/'
SAVE_PATH = './feats/'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)

print('process_user_info...')

process_user_info...


In [3]:
user_info = pd.read_pickle(os.path.join(DATA_PATH, 'user_info.pkl'))

In [4]:
# 原始类别特征编码
user_cat_feats = ['gender', 'freq', 'A1', 'B1', 'C1', 'D1', 'E1', 'A2', 'B2', 'C2', 'D2', 'E2']
for feat in user_cat_feats:
    user_info[feat] = LabelEncoder().fit_transform(user_info[feat])

In [5]:
# 用户关注和感兴趣的topic数
user_info['num_topic_a'] = user_info['topic_a'].progress_apply(len)
user_info['num_topic_i'] = user_info['topic_i'].progress_apply(len)

100%|██████████| 1931654/1931654 [00:02<00:00, 932923.91it/s] 


In [6]:
# 用户最感兴趣的topic
def most_interest_topic(d):
    if len(d) == 0:
        return -1
    return max(d, key=d.get)
user_info['most_topic_i'] = user_info['topic_i'].progress_apply(most_interest_topic)

100%|██████████| 1931654/1931654 [00:04<00:00, 470315.28it/s]


In [7]:
# 用户topic兴趣值的统计特征
user_info['topic_iv'] = user_info['topic_iv'].progress_apply(lambda x: x if len(x) > 0 else [0])
user_info['min_topic_iv'] = user_info['topic_iv'].progress_apply(np.min)
user_info['max_topic_iv'] = user_info['topic_iv'].progress_apply(np.max)
user_info['mean_topic_iv'] = user_info['topic_iv'].progress_apply(np.mean)
user_info['std_topic_iv'] = user_info['topic_iv'].progress_apply(np.std)

100%|██████████| 1931654/1931654 [01:07<00:00, 28616.61it/s]


In [8]:
user_info.to_pickle(os.path.join(DATA_PATH, 'user_info_p.pkl'))

In [9]:
toc = time.time()
print('Used time: %d' % int(toc-tic))

Used time: 185
